In [ ]:
import sys
from PyQt5 import QtWidgets, uic, QtGui, QtCore
from PyQt5.QtWidgets import QTableWidget, QTableWidgetItem, QHeaderView, QLabel, QMessageBox, QDialog, QVBoxLayout, QLineEdit, QTextEdit, QDialogButtonBox
from PyQt5.QtWidgets import QPushButton, QGraphicsDropShadowEffect, QProgressDialog, QButtonGroup, QRadioButton, QHBoxLayout
from PyQt5.QtGui import QColor, QBrush, QPainter, QPixmap
from PyQt5.QtCore import Qt, QTimer, QRect, QThread, pyqtSignal, QSettings
import cx_Oracle
from datetime import datetime
import socket
import math
import ftplib
import tempfile
import subprocess
import re
## vega 접속
import pyodbc
import os
from RPA_UI import Ui_TOTAL


class LoadingOverlay(QLabel):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAlignment(Qt.AlignCenter)
        self.setText("Loading...")
        self.setStyleSheet("""
            background-color: rgba(0, 0, 0, 150);
            color: white;
            font-size: 24px;
            font-weight: bold;
        """)
        self.angle = 0
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.rotate)
        self.timer.start(50)

    def rotate(self):
        self.angle = (self.angle + 10) % 360
        self.update()

    def paintEvent(self, event):
        super().paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        
        center = self.rect().center()
        painter.translate(center)
        painter.rotate(self.angle)
        
        painter.setPen(Qt.NoPen)
        painter.setBrush(QColor(255, 255, 255))
        
        for i in range(8):
            painter.rotate(45)
            painter.drawEllipse(-5, -30, 10, 10)


class FTPUpdateChecker(QThread):
    update_available = pyqtSignal(str, str, str)  # 버전, 파일명, 변경사항
    check_completed = pyqtSignal(bool)  # 확인 완료 여부
    
    def __init__(self):
        super().__init__()
        self.current_version = "4.1.1"  # 현재 프로그램 버전
        self.ftp_host = "192.168.223.225"
        self.ftp_user = "vega"
        self.ftp_pass = "vegagcc"
        self.ftp_path = "/"
    
    def run(self):
        print(f"현재 설정된 버전: {self.current_version}")
        try:
            # FTP 연결
            ftp = ftplib.FTP(self.ftp_host)
            ftp.login(self.ftp_user, self.ftp_pass)
            
            # 경로가 루트가 아니면 해당 디렉토리로 이동
            if self.ftp_path != "/":
                ftp.cwd(self.ftp_path)
            
            # version.txt 파일 존재 확인 및 읽기
            try:
                lines = []
                ftp.retrlines('RETR version.txt', lines.append)
                
                if lines:
                    # 첫 번째 줄에서 버전 정보 추출
                    version_info = lines[0].strip()
                    print(f"FTP에서 가져온 버전 정보: {version_info}")
                    
                    # 버전 정보 파싱 (예: "Version: 4.1.2, File: update.exe, Changes: 버그수정")
                    if version_info.startswith("Version:"):
                        parts = version_info.split(", ")
                        if len(parts) >= 3:
                            ftp_version = parts[0].split(":")[1].strip()
                            file_name = parts[1].split(":")[1].strip()
                            changes = parts[2].split(":")[1].strip()
                            
                            print(f"파싱된 정보 - 버전: {ftp_version}, 파일: {file_name}, 변경사항: {changes}")
                            
                            # 버전 비교 (간단한 버전 비교)
                            if self.is_newer_version(ftp_version):
                                print("새 버전이 발견되었습니다!")
                                self.update_available.emit(ftp_version, file_name, changes)
                            else:
                                print("현재 버전이 최신입니다.")
                        else:
                            print("버전 정보 형식이 올바르지 않습니다.")
                    else:
                        print("버전 정보 형식이 올바르지 않습니다.")
                else:
                    print("version.txt 파일이 비어있습니다.")
            except Exception as e:
                print(f"version.txt 파일을 읽을 수 없습니다: {e}")
            
            ftp.quit()
            self.check_completed.emit(True)
            
        except Exception as e:
            print(f"FTP 연결 오류: {e}")
            self.check_completed.emit(False)
    
    def is_newer_version(self, ftp_version):
        # 간단한 버전 비교 (예: "4.1.2" > "4.1.1")
        try:
            current_parts = list(map(int, self.current_version.split('.')))
            ftp_parts = list(map(int, ftp_version.split('.')))
            
            # 길이를 맞춰줌
            max_len = max(len(current_parts), len(ftp_parts))
            current_parts.extend([0] * (max_len - len(current_parts)))
            ftp_parts.extend([0] * (max_len - len(ftp_parts)))
            
            return ftp_parts > current_parts
        except:
            return False


class MainWindow(QtWidgets.QMainWindow, Ui_TOTAL):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.setupUi(self)
        self.selected_campus = None  # Add campus selection tracking
        self.current_action = None
        self.loading_overlay = LoadingOverlay(self)
        self.loading_overlay.hide()
        self.update_checker = None
        
        # UI 요소 찾기
        self.findUIElements()
        
        # 창을 화면 중앙에 배치
        self.center()
        
        # # 시작 시 숨김 요소들
        # self.grid_layout_2_widget.hide()
        # self.tabWidget.hide()
        
        # # 원래 화면으로 돌아가는 버튼
        # self.back_to_main_button.clicked.connect(self.showMainScreen)
        
        # Fetch_Data 버튼 이벤트 연결
        # self.FETCH_DATA.clicked.connect(self.fetchDataAndShowTables)
        
        # 설정 로드
        self.loadSettings()
        
        # 메인 화면을 먼저 보여줌
        self.showMainScreen()
        
        # 이벤트 연결 (중복 제거를 위해 마지막에)
        self.connectEvents()
        
        # 스타일 적용
        self.setStyleSheet("""
            QMainWindow {
                background-color: #f0f0f0;
            }
            QTabWidget::pane {
                border: 2px solid #C2C7CB;
                background-color: white;
            }
            QTabWidget::tab-bar {
                left: 5px;
            }
            QTabBar::tab {
                background: #E1E1E1;
                border: 2px solid #C4C4C3;
                border-bottom-color: #C2C7CB;
                border-top-left-radius: 4px;
                border-top-right-radius: 4px;
                min-width: 8ex;
                padding: 8px;
                margin-right: 2px;
            }
            QTabBar::tab:selected {
                background: white;
                border-color: #9B9B9B;
                border-bottom-color: white;
            }
            QTabBar::tab:!selected {
                margin-top: 2px;
            }
        """)
        
        # 프로그램 시작 시 자동 업데이트 확인
        self.check_for_updates()

    def center(self):
        qr = self.frameGeometry()
        cp = QtWidgets.QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())

    def findUIElements(self):
        self.tableWidget = self.findChild(QTableWidget, "tableWidget")
        self.closeTableButton = self.findChild(QPushButton, "close_table")
        
        # 닫기 버튼 연결
        self.closeTableButton.clicked.connect(self.onCloseButtonClicked)
        
        self.STATUS_ID = self.findChild(QLabel, "STATUS_ID")
        
        self.grid_layout_2_widget = self.findChild(QtWidgets.QWidget, "widget_3")
        self.back_to_main_button = self.findChild(QPushButton, "back_to_main")
        
        self.label_5 = self.findChild(QLabel, "label_5")
        
        # HOME 버튼 연결
        self.PUSH_HOME = self.findChild(QtWidgets.QPushButton, "PUSH_HOME")
        self.PUSH_HOME.clicked.connect(self.switchToMainTab)

        self.tabWidget = self.findChild(QtWidgets.QTabWidget, "tabWidget") 
        
        
        self.PUSH_LOT_CHANGE = self.findChild(QtWidgets.QPushButton, "PUSH_LOT_CHANGE")
        self.PUSH_LOT_CHANGE.clicked.connect(lambda: self.showTable("LOT_CHANGE"))
        
        self.PUSH_CONVERSION = self.findChild(QtWidgets.QPushButton, "PUSH_CONVERSION")
        self.PUSH_CONVERSION.clicked.connect(lambda: self.showTable("CONVERSION"))      
        
        self.PUSH_LAST_DIE = self.findChild(QtWidgets.QPushButton, "PUSH_LAST_DIE")
        self.PUSH_LAST_DIE.clicked.connect(lambda: self.showTable("LAST_DIE"))
        
        self.PUSH_RETEST = self.findChild(QtWidgets.QPushButton, "PUSH_RETEST")
        self.PUSH_RETEST.clicked.connect(lambda: self.showTable("RETEST"))
        
        self.RPA_START_3 = self.findChild(QtWidgets.QPushButton, "RPA_START_3")
        self.RPA_START_3.clicked.connect(self.insert_data)  
        
        self.PUSH_MONITORING_DATA = self.findChild(QtWidgets.QPushButton, "PUSH_MONITORING_DATA") 
        self.PUSH_MONITORING_DATA.clicked.connect(lambda: self.showTable("MONITORING"))
        
        self.PUSH_USER_INFO = self.findChild(QtWidgets.QPushButton, "PUSH_USER_INFO")
        self.PUSH_USER_INFO.clicked.connect(self.showUserAndKeyDialog)
        
        # FTP 업데이트 시스템 초기화
        self.setup_update_system()
        
        if self.grid_layout_2_widget:
            self.grid_layout_2_widget.hide()
        
        # 닫기 버튼 스타일
        self.styleCloseButton()
        
        self.back_to_main_button = self.findChild(QPushButton, "back_to_main")
        self.back_to_main_button.clicked.connect(self.showMainScreen)

    def connectEvents(self):
        # 이미 findUIElements에서 연결했으므로 중복 연결 방지
        pass

    def loadSettings(self):
        settings = QSettings('MyCompany', 'MyApp')
        # 설정이 있으면 로드, 없으면 기본값 사용
        pass

    def showMainScreen(self):
        """메인 화면을 보여주는 함수"""
        if hasattr(self, 'tabWidget'):
            self.tabWidget.hide()
        if hasattr(self, 'grid_layout_2_widget'):
            self.grid_layout_2_widget.hide()
        
        # 기본 UI 요소들을 보여줌
        # 필요에 따라 여기에 메인 화면 요소들 show() 추가

    def switchToMainTab(self):
        """HOME 버튼 클릭 시 메인 탭으로 전환"""
        if self.tabWidget:
            self.tabWidget.setCurrentIndex(0)  # 첫 번째 탭으로 전환

    def resizeEvent(self, event):
        super().resizeEvent(event)
        if hasattr(self, 'loading_overlay'):
            self.loading_overlay.setGeometry(self.rect())

    def setup_update_system(self):
        """FTP 업데이트 시스템 초기화"""
        # UI에서 업데이트 관련 버튼들을 찾습니다
        self.update_button = self.findChild(QPushButton, "update_button")  # 실제 버튼 이름으로 변경
        if self.update_button:
            self.update_button.clicked.connect(self.check_for_updates)
    
    def check_for_updates(self):
        """업데이트 확인을 시작하는 함수"""
        print("업데이트 확인을 시작합니다...")
        
        if self.update_checker is None or not self.update_checker.isRunning():
            self.update_checker = FTPUpdateChecker()
            self.update_checker.update_available.connect(self.on_update_available)
            self.update_checker.check_completed.connect(self.on_check_completed)
            self.update_checker.start()
        else:
            print("이미 업데이트 확인이 진행 중입니다.")
    
    def on_update_available(self, version, filename, changes):
        """새 버전이 발견되었을 때 호출되는 함수"""
        print(f"새 버전 발견: {version}")
        reply = QMessageBox.question(
            self, 
            '업데이트 알림',
            f'새로운 버전이 있습니다!\n\n'
            f'현재 버전: {self.update_checker.current_version}\n'
            f'새 버전: {version}\n'
            f'파일명: {filename}\n'
            f'변경사항: {changes}\n\n'
            f'업데이트를 진행하시겠습니까?',
            QMessageBox.Yes | QMessageBox.No,
            QMessageBox.Yes
        )
        
        if reply == QMessageBox.Yes:
            self.download_and_install_update(filename)
    
    def on_check_completed(self, success):
        """업데이트 확인 완료 시 호출되는 함수"""
        if success:
            print("업데이트 확인이 완료되었습니다.")
        else:
            print("업데이트 확인 중 오류가 발생했습니다.")
    
    def download_and_install_update(self, filename):
        """업데이트 파일을 다운로드하고 설치하는 함수"""
        try:
            print(f"업데이트 파일 다운로드 시작: {filename}")
            
            # 임시 디렉토리에 파일 다운로드
            temp_dir = tempfile.gettempdir()
            local_path = os.path.join(temp_dir, filename)
            
            # FTP에서 파일 다운로드
            ftp = ftplib.FTP(self.update_checker.ftp_host)
            ftp.login(self.update_checker.ftp_user, self.update_checker.ftp_pass)
            
            if self.update_checker.ftp_path != "/":
                ftp.cwd(self.update_checker.ftp_path)
            
            with open(local_path, 'wb') as local_file:
                ftp.retrbinary(f'RETR {filename}', local_file.write)
            
            ftp.quit()
            
            print(f"다운로드 완료: {local_path}")
            
            # 다운로드된 파일 실행
            QMessageBox.information(
                self,
                '업데이트 다운로드 완료',
                f'업데이트 파일이 다운로드되었습니다.\n\n'
                f'파일 위치: {local_path}\n\n'
                f'프로그램을 종료하고 업데이트를 진행하세요.'
            )
            
            # 업데이트 파일 실행 (선택적)
            reply = QMessageBox.question(
                self,
                '업데이트 실행',
                '지금 업데이트를 실행하시겠습니까?\n(현재 프로그램이 종료됩니다)',
                QMessageBox.Yes | QMessageBox.No,
                QMessageBox.No
            )
            
            if reply == QMessageBox.Yes:
                # 업데이트 파일 실행 후 현재 프로그램 종료
                subprocess.Popen([local_path])
                self.close()
                
        except Exception as e:
            print(f"업데이트 다운로드 오류: {e}")
            QMessageBox.critical(
                self,
                '업데이트 오류',
                f'업데이트 다운로드 중 오류가 발생했습니다:\n{str(e)}'
            )

    def fetchDataAndShowTables(self):
        """데이터를 가져와서 테이블들을 보여주는 함수"""
        # 로딩 화면 표시
        self.loading_overlay.setGeometry(self.rect())
        self.loading_overlay.show()
        
        # 100ms 후에 실제 데이터 로딩 작업 시작
        QTimer.singleShot(100, self.executeQuery)

    def clearData(self):
        """Clear data from the interface table"""
        try:
            # 데이터베이스 연결
            dsn_tns = cx_Oracle.makedsn('192.168.223.13', 1521, service_name='ERPSIMAX')
            conn = cx_Oracle.connect(user='RPA', password='rpa01!', dsn=dsn_tns)
            cursor = conn.cursor()
    
            # 오늘 날짜의 데이터만 삭제하는 쿼리
            delete_query = """
                DELETE FROM RPA_ADMIN.interface
                WHERE TRUNC(DATE_) = TRUNC(SYSDATE)
            """
            cursor.execute(delete_query)
            conn.commit()
    
            cursor.close()
            conn.close()
    
            # 성공적으로 데이터를 삭제한 후 UI 업데이트
            self.updateStatus()
            print("Today's data has been successfully cleared.")
    
        except Exception as e:
            print(f"Error clearing data: {e}")
            QMessageBox.critical(self, "오류", f"데이터 삭제 중 오류가 발생했습니다: {str(e)}")

    def updateStatus(self):
        # 상태를 업데이트하고 테이블을 다시 로드합니다
        if hasattr(self, 'tableWidget'):
            self.tableWidget.setRowCount(0)  # 테이블 내용을 지웁니다
            
        if hasattr(self, 'label_5'):
            self.label_5.show()

        # 데이터가 성공적으로 지워졌다는 메시지를 표시합니다
        QtWidgets.QMessageBox.information(self, "알림", "Data clearing was successful.")
        
    def showTable(self, action_type):
        """Show campus selection dialog first, then proceed with table display"""
        campus = self.showCampusSelectionDialog()
        if campus:
            self.selected_campus = campus
            self.showTableWithCampus(action_type)
    
    def showCampusSelectionDialog(self):
        """Create and show campus selection dialog"""
        dialog = QDialog(self)
        dialog.setWindowTitle("캠퍼스 선택")
        dialog.setFixedSize(300, 150)
        
        layout = QVBoxLayout()
        
        # Title label
        title_label = QLabel("캠퍼스를 선택해주세요:")
        title_label.setStyleSheet("font-weight: bold; margin-bottom: 10px;")
        layout.addWidget(title_label)
        
        # Radio buttons for campus selection
        button_group = QButtonGroup()
        radio_layout = QHBoxLayout()
        
        ns2_radio = QRadioButton("NS2")
        ns3_radio = QRadioButton("NS3")
        
        button_group.addButton(ns2_radio)
        button_group.addButton(ns3_radio)
        
        radio_layout.addWidget(ns2_radio)
        radio_layout.addWidget(ns3_radio)
        layout.addLayout(radio_layout)
        
        # OK and Cancel buttons
        button_box = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel)
        button_box.accepted.connect(dialog.accept)
        button_box.rejected.connect(dialog.reject)
        layout.addWidget(button_box)
        
        dialog.setLayout(layout)
        
        result = dialog.exec_()
        
        if result == QDialog.Accepted:
            if ns2_radio.isChecked():
                return "NS2"
            elif ns3_radio.isChecked():
                return "NS3"
            else:
                QMessageBox.warning(self, "경고", "캠퍼스를 선택해주세요.")
                return None
        else:
            return None
    
    def showTableWithCampus(self, action_type):
        """Original showTable logic with campus support"""
        self.current_action = action_type  # 현재 액션 저장
        self.STATUS_ID.setText(action_type)  # Monitor 탭의 STATUS_ID에 텍스트 설정
        self.loading_overlay.setGeometry(self.rect())
        self.loading_overlay.show()
        QTimer.singleShot(100, self.executeQuery)
        self.closeTableButton.show()
    
    def onCloseButtonClicked(self):
        if self.grid_layout_2_widget:
            self.grid_layout_2_widget.hide()
        if self.tableWidget.isVisible():
            self.tableWidget.show()
            self.closeTableButton.show()
        else:
            self.tableWidget.hide()
            self.closeTableButton.hide()

    def styleCloseButton(self):
        # 닫기 버튼 스타일링
        self.closeTableButton.setStyleSheet("""
            QPushButton {
                background-color: #ff6b6b;
                border: none;
                color: white;
                font-size: 14px;
                font-weight: bold;
                border-radius: 20px;
                min-width: 80px;
                min-height: 40px;
            }
            QPushButton:hover {
                background-color: #ff5252;
            }
            QPushButton:pressed {
                background-color: #ff4444;
            }
        """)
        
        # 그림자 효과 추가
        shadow = QGraphicsDropShadowEffect()
        shadow.setBlurRadius(10)
        shadow.setColor(QColor(0, 0, 0, 100))
        shadow.setOffset(2, 2)
        
        # 버튼에 그림자 효과 적용
        self.closeTableButton.setGraphicsEffect(shadow)
        
        
    def executeQuery(self):
        # 데이터베이스 연결 정보 수정
        # dsn_tns = cx_Oracle.makedsn('192.168.223.13', 1521, service_name='ERPSIMAX')
        # connection = cx_Oracle.connect(user='RPA', password='rpa01!', dsn=dsn_tns)
        # cursor = connection.cursor()
        
        dsn = cx_Oracle.makedsn("mes.nepes.co.kr", 1521, "CCUBE")
        connection = cx_Oracle.connect("mighty", "mighty", dsn)
        cursor = connection.cursor()
    
        #  수정가능 -->   RPA_ADMIN.view_eqpsts
        # DB
        query = """
            SELECT *
                FROM(
                    SELECT
                    s.PLANT, s.EQUIPMENT_ID, s.MAIN_STATUS, s.USER_COMMENT, s.TRANS_TIME,
                    CASE
                        /* T2K -NS3 */
                        WHEN s.EQUIPMENT_ID IN (
                            'T2K75','T2K76','T2K85','T2K93','T2K94','T2K95','T2K98','T2K99',
                            'T2K100','T2K101','T2K102','T2K103','T2K104','T2K105','T2K106'
                        ) THEN 'NS3'
                        /* T2K-NS2 */
                        WHEN s.EQUIPMENT_ID LIKE '%T2K%' THEN 'NS2'
                        /* U-FLEX-NS2 */
                        WHEN s.EQUIPMENT_ID IN (
                            'UTS04', 'UTS05', 'UTS06', 'UTS07', 'UTS08', 'UTSP08', 'UTS09', 'UTSP09', 'UTS10') THEN 'NS2'
                        /* U-FLEX-XD */  
                        WHEN s.EQUIPMENT_ID LIKE '%UTS%' THEN 'NS3'
                        /* U-FLEX-PLUS */
                        WHEN s.EQUIPMENT_ID LIKE '%UTC%' THEN 'NS3'
                        /* ETS600 */
                        WHEN s.EQUIPMENT_ID LIKE '%ETS%'
                        AND LENGTH(s.EQUIPMENT_ID) = 5 THEN 'NS2'
                        /*V93K */
                        WHEN s.EQUIPMENT_ID LIKE '%V93K%' THEN 'NS3'
                        /*ND4 */
                        WHEN s.EQUIPMENT_ID LIKE '%ND4%' THEN 'NS3'
                        /*DIAMONDX */
                        WHEN s.EQUIPMENT_ID LIKE '%DMX%' THEN 'NS3'
                        /* 일치하는 것이 없는 경우: NULL 반환 */
                        ELSE NULL
                    END AS CAMPUS
                FROM (
                    SELECT
                        e.*,
                        ROW_NUMBER() OVER (
                            PARTITION BY e.EQUIPMENT_ID
                            ORDER BY TO_DATE(e.TRANS_TIME, 'YYYYMMDD HH24MISS') DESC
                        ) AS rn
                    FROM EQPSTS e
                ) s
                WHERE s.rn = 1
                ) x

                WHERE x.CAMPUS IS NOT NULL
                AND x.PLANT IN 'CCUBEDIGITAL'
                AND x.MAIN_STATUS NOT IN ('RUN', 'SHUTDOWN')
                AND x.CAMPUS = :campus
 
            """
        
    
        cursor.execute(query, {'campus': self.selected_campus})
        results = sorted(cursor.fetchall(), key=lambda x: x[2] if x[2] is not None else '')
    
        # self.tableWidget.setRowCount(len(results))
        # for i, row in enumerate(results):
        #     for j, value in enumerate(row):
        #         item = QTableWidgetItem(str(value))
        #         item.setTextAlignment(Qt.AlignCenter)
        #         self.tableWidget.setItem(i, j, item)
        
        self.tableWidget.setRowCount(len(results))

        for i, row in enumerate(results):

            main_status = row[2]  # MAIN_STATUS 컬럼 (인덱스 2)
            
            # MAIN_STATUS에 따른 색상 설정
            main_status_str = str(main_status).upper() if main_status else ''
            
            if 'ERROR' in main_status_str:
                # ERROR 상태일 때 빨간색
                row_color = QColor(255, 182, 193)  # Light pink
            elif 'IDLE' in main_status_str:
                # IDLE 상태일 때 노란색
                row_color = QColor(255, 255, 224)  # Light yellow
            else:
                # 기본 색상 (흰색)
                row_color = QColor(255, 255, 255)  # White

            for j, value in enumerate(row):
                item = QTableWidgetItem(str(value))
                item.setTextAlignment(Qt.AlignCenter)
                
                # 배경색 설정
                item.setBackground(QBrush(row_color))
                
                self.tableWidget.setItem(i, j, item)

        # 컬럼 너비 자동 조정
        header = self.tableWidget.horizontalHeader()
        header.setSectionResizeMode(QHeaderView.ResizeToContents)
        
        # 로딩 화면 숨김
        self.loading_overlay.hide()
        
        # 테이블과 닫기 버튼 표시
        self.tableWidget.show()
        self.closeTableButton.show()
        
        cursor.close()
        connection.close()

    def get_lot_id(self, row, col):
        # 테이블 셀이 클릭되었을 때 해당 행의 LOT_ID를 가져옴
        item = self.tableWidget.item(row, 1)  # LOT_ID는 두 번째 컬럼(인덱스 1)에 있다고 가정
        if item:
            return item.text()
        return None

    def on_table_cell_double_clicked(self, row, column):
        # LOT_ID 가져오기
        lot_id = self.get_lot_id(row, column)
        if not lot_id:
            return

        try:
            # 오라클 DB 연결
            dsn = cx_Oracle.makedsn("mes.nepes.co.kr", 1521, "CCUBE")
            connection = cx_Oracle.connect("mighty", "mighty", dsn)
            cursor = connection.cursor()

            # 쿼리 실행
            query = """
                SELECT *
                FROM (
                    SELECT 
                        EQP_ID, 
                        LOT_ID,  
                        STEP_ID,  
                        STATUS,
                        ROW_NUMBER() OVER (
                            PARTITION BY LOT_ID, EQP_ID, STEP_ID 
                            ORDER BY DATE_ DESC
                        ) AS rn
                    FROM MIGHTY.EQP_STATUS  
                    WHERE LOT_ID = :lot_id
                )
                WHERE rn = 1
                ORDER BY STEP_SEQ ASC
                """

                cursor.execute(query, {'lot_id': lot_id})
                results = cursor.fetchall()

                # 'STATUS' 컬럼값에 'RUN'이 포함되어 있을 때 인터락 추가
                if any('RUN' in row[3] for row in results):
                    QMessageBox.warning(self, "경고", "해당 LOT_ID는 현재 'RUN' 상태입니다.")
                    cursor.close()
                    connection.close()
                    return

                # 창 생성 후 결과 표시
                self.show_lot_details(results)

                cursor.close()
                connection.close()

        except Exception as e:
            QMessageBox.critical(self, "오류", f"데이터를 가져오는 중 오류가 발생했습니다: {str(e)}")

    def show_lot_details(self, results):
        # 팝업 창 생성
        dialog = QtWidgets.QDialog(self)
        dialog.setWindowTitle("LOT 상세 정보")
        dialog.resize(800, 600)

        # 레이아웃 설정
        layout = QtWidgets.QVBoxLayout()

        # 테이블 위젯 생성
        table = QtWidgets.QTableWidget()
        table.setColumnCount(4)
        table.setHorizontalHeaderLabels(["EQP_ID", "LOT_ID", "STEP_ID", "STATUS"])
        table.setRowCount(len(results))

        # 데이터 채우기
        for i, row in enumerate(results):
            for j, value in enumerate(row):
                item = QtWidgets.QTableWidgetItem(str(value))
                item.setTextAlignment(Qt.AlignCenter)
                table.setItem(i, j, item)

        # 컬럼 너비 조정
        header = table.horizontalHeader()
        header.setSectionResizeMode(QtWidgets.QHeaderView.ResizeToContents)

        # 레이아웃에 추가
        layout.addWidget(table)

        # 닫기 버튼
        close_button = QtWidgets.QPushButton("닫기")
        close_button.clicked.connect(dialog.close)
        layout.addWidget(close_button)

        dialog.setLayout(layout)
        dialog.exec_()

    def showUserAndKeyDialog(self):
        # 사용자 정보와 키 정보를 입력받는 대화상자
        dialog = QDialog(self)
        dialog.setWindowTitle("사용자 및 키 정보 입력")
        dialog.setFixedSize(400, 600)
        
        layout = QVBoxLayout()
        
        # PRODUCT 입력
        product_layout = QVBoxLayout()
        product_label = QLabel("PRODUCT (앞 7자리):")
        product_input = QLineEdit()
        product_input.setPlaceholderText("예: NS2K001")
        product_layout.addWidget(product_label)
        product_layout.addWidget(product_input)
        layout.addLayout(product_layout)
        
        # Probe Card ID 입력
        probe_layout = QVBoxLayout()
        probe_label = QLabel("Probe Card ID:")
        probe_input = QLineEdit()
        probe_input.setPlaceholderText("예: NS2K001-001")
        probe_layout.addWidget(probe_label)
        probe_layout.addWidget(probe_input)
        layout.addLayout(probe_layout)
        
        # 조회 버튼
        query_button = QPushButton("조회")
        layout.addWidget(query_button)
        
        # 결과 표시를 위한 텍스트 영역
        result_text = QTextEdit()
        result_text.setReadOnly(True)
        layout.addWidget(result_text)
        
        def perform_query():
            product_prefix = product_input.text().strip()
            probe_card_id = probe_input.text().strip()
            
            if not product_prefix or not probe_card_id:
                QMessageBox.warning(dialog, "경고", "PRODUCT와 Probe Card ID를 모두 입력해주세요.")
                return
            
            try:
                # 데이터베이스 연결
                dsn = cx_Oracle.makedsn("mes.nepes.co.kr", 1521, "CCUBE")
                connection = cx_Oracle.connect("mighty", "mighty", dsn)
                cursor = connection.cursor()
                
                # SQL 쿼리
                query = """
                SELECT DISTINCT B.PC_ID
                FROM EDS_DB.PMS_MAIN_PC B
                WHERE SUBSTR(B.PROD, 1, 7) = :product_prefix
                
                ORDER BY PC_ID ASC
                """

                cursor.execute(query, {'product_prefix': product_prefix})
                results = cursor.fetchall()

                # 입력된 Probe Card ID와 비교하여 필터링
                filtered_results = [row for row in results if row[0] == probe_card_id]

                result_text.clear()
                if filtered_results:
                    result_text.append("검색 결과:")
                    result_text.append("-" * 50)
                    for row in filtered_results:
                        result_text.append(f"PC_ID: {row[0]}")
                    result_text.append(f"\n총 {len(filtered_results)}개의 결과가 있습니다.")
                    
                    # PIB 정보도 함께 조회
                    self.query_pib_info(product_prefix, result_text, cursor)
                else:
                    result_text.append("해당 조건에 맞는 Probe Card를 찾을 수 없습니다.")

                cursor.close()
                connection.close()
                
            except Exception as e:
                result_text.clear()
                result_text.append(f"오류가 발생했습니다: {str(e)}")
        
        query_button.clicked.connect(perform_query)
        
        # 대화상자 버튼
        button_box = QDialogButtonBox(QDialogButtonBox.Close)
        button_box.rejected.connect(dialog.reject)
        layout.addWidget(button_box)
        
        dialog.setLayout(layout)
        dialog.exec_()

    def query_pib_info(self, product_prefix, result_text, cursor):
        """PIB 정보를 조회하고 결과에 추가하는 함수"""
        try:
            # PIB 정보 조회
            query = """
            SELECT DISTINCT A.PC_ID, A.EQP_ID, A.PARA
            FROM EDS_DB.PMS_MAIN_PB A
            WHERE SUBSTR(A.PROD, 1, 7) = :product_prefix
            AND REGEXP_LIKE(A.PC_ID, '-[0-9]{3}$')
            """

            cursor.execute(query, {'product_prefix': product_prefix})
            results = cursor.fetchall()

            pib_model = QtGui.QStandardItemModel()
            pib_model.setHorizontalHeaderLabels(["PIB_ID", "설비", "PARA"])
            for row in results:
                items = [QtGui.QStandardItem(str(item)) for item in row]
                pib_model.appendRow(items)

            if results:
                result_text.append("\n" + "=" * 50)
                result_text.append("PIB 정보:")
                result_text.append("=" * 50)
                for row in results:
                    result_text.append(f"PIB_ID: {row[0]}, 설비: {row[1]}, PARA: {row[2]}")
                result_text.append(f"\n총 {len(results)}개의 PIB 정보가 있습니다.")
            else:
                result_text.append("\n해당 조건에 맞는 PIB 정보를 찾을 수 없습니다.")
                
        except Exception as e:
            result_text.append(f"\nPIB 정보 조회 중 오류 발생: {str(e)}")

    def insert_data(self):
        try:
            # 현재 시간 가져오기
            current_time = datetime.now().strftime('%Y%m%d%H%M%S')
            
            # IP 주소 가져오기
            hostname = socket.gethostname()
            local_ip = socket.gethostbyname(hostname)
            
            # 예시 데이터 (실제로는 UI에서 가져와야 함)
            eqp_id = "TEST_EQP_001"
            device = "TEST_DEVICE"
            lot_id = "TEST_LOT_001"  
            step_id = "TEST_STEP_001"
            pc_id = "TEST_PC_001"
            pb_id = "TEST_PB_001"
            user_id = "TEST_USER"
            status_id = "INSERT"
            
            # 오라클 DB 연결 (ERPSIMAX)
            dsn_tns = cx_Oracle.makedsn('192.168.223.13', 1521, service_name='ERPSIMAX')
            conn = cx_Oracle.connect(user='RPA', password='rpa01!', dsn=dsn_tns)
            cursor = conn.cursor()

            # 테이블명 수정 및 데이터 삽입
            cursor.execute("""
                INSERT INTO RPA_ADMIN.interface 
                (EQPID, DATE_, DEVICE, LOT_ID, STEP_ID, PC_ID, PB_ID, USER_ID, IP, STATUS)
                VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10)
            """, (eqp_id, current_time, device, lot_id, step_id, pc_id, pb_id, user_id, local_ip, status_id))
            
            conn.commit()
            cursor.close()
            conn.close()
            
            # 성공 메시지
            QtWidgets.QMessageBox.information(self, "성공", f"데이터가 성공적으로 삽입되었습니다.\n시간: {current_time}")
            
        except Exception as e:
            QtWidgets.QMessageBox.critical(self, "오류", f"데이터 삽입 중 오류가 발생했습니다: {str(e)}")

    def fetch_user_data(self):
        """사용자 데이터를 가져오는 함수"""
        try:
            # Vega 서버에 연결
            server = 'DH-VEGA'
            database = 'VegaEMS'
            username = 'sa'
            password = 'emsems!@#$'
            
            # 연결 문자열 생성
            conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
            
            # DB 연결
            conn = pyodbc.connect(conn_str)
            cursor = conn.cursor()
    
            # 쿼리 실행
            query = """
                SELECT USER_ID
                FROM userinfo
                WHERE gw_dept_name LIKE '%하이퍼%'
            """
            cursor.execute(query)
            results = cursor.fetchall()
    
            # 디버깅용 출력
            print("Query Results:", results)
    
            # 연결 종료
            cursor.close()
            conn.close()
    
            return [row[0] for row in results]
    
        except Exception as e:
            print(f"Database connection error: {e}")
            QMessageBox.critical(self, "데이터베이스 오류", f"사용자 데이터를 가져올 수 없습니다: {str(e)}")
            return []

    def open_file_dialog(self):
        """파일 선택 대화상자를 여는 함수"""
        file_path, _ = QtWidgets.QFileDialog.getOpenFileName(
            self,
            "설정 파일 선택",
            "",
            "설정 파일 (*.ini *.cfg *.conf);;모든 파일 (*)"
        )
        
        if file_path:
            self.label_4.setText(f"선택된 파일: {file_path}")
            # 여기에 파일 처리 로직 추가
            self.process_selected_file(file_path)
    
    def process_selected_file(self, file_path):
        """선택된 파일을 처리하는 함수"""
        try:
            # 파일 처리 로직 구현
            print(f"Processing file: {file_path}")
            # 실제 파일 처리 코드를 여기에 추가
            
        except Exception as e:
            QMessageBox.critical(self, "파일 처리 오류", f"파일을 처리하는 중 오류가 발생했습니다: {str(e)}")

if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())